In [1]:
import torch 
from src.train_module import ModelTrainer
from src.balu import BaLu
from argparse import Namespace
from src.train_utils import filter_dataset

full_data_path = '/Users/jason/Documents/Coding Projects/2025_Claude/NetDeconf_main_hao/datasets/exps/BlogCatalog/p=0.0_k=9_seed=194.pt'
# full_data_path = 'datasets/exps/Syn/p=0.0_k=0_seed=919.pt'

data = torch.load(full_data_path, weights_only=False)

seed = 460
params = {
    'dataset': 'Syn',         # REQUIRED: replace with actual dataset name
    'missing_p': 0.1,                        # REQUIRED: replace with actual missing percentage

    'model_name': 'BaLu_Ultra',

    'gconv': 'GraphSAGE',
    'rconv': 'RGCN',

    'imputer': 'BaLu_IGMC',
    'imputer_node_dims': [64, 64, 32],

    'edge_dim': 16,
    'dropout': 0.1,
    'rel_dropout': 0.2,

    'interference': 'GNN',
    'interference_node_dims': [64, 32],

    'outcome_rep': 'h_r+X*+h_t',

    'lr': 1e-3,
    'weight_decay': 1e-4,
    'n_epochs': 2000,

    'norm_y': True,

    'alpha': 1.0,
    'beta': 1e-4,
    'gamma': 1e-4,
    'eta': 1e-4,

    'early_stop': True,
    'patience': 25,
}
# params = Namespace(**params)

trainer = ModelTrainer(BaLu, params, seed=seed)

train_data = filter_dataset(data, data.val_mask | data.train_mask)
print("star train")
best_model = trainer.train(train_data)# data)
print("finished train")
results = trainer.evaluate(data, unit_indexes=data.test_mask)
print("Evaluation Results:", results)


star train
train_loss: 1.06930673122406; val_loss: 1.0504820346832275
train_loss: 1.045647382736206; val_loss: 1.0294960737228394
train_loss: 1.0414314270019531; val_loss: 1.0064090490341187
train_loss: 0.9951024055480957; val_loss: 0.9971717000007629
train_loss: 0.9847655296325684; val_loss: 0.9799805283546448
train_loss: 0.9856551289558411; val_loss: 0.9777274131774902
train_loss: 0.9899980425834656; val_loss: 0.9717223048210144
train_loss: 0.9629067778587341; val_loss: 0.9561530351638794
train_loss: 0.9717110395431519; val_loss: 0.9413155913352966
train_loss: 0.9685677886009216; val_loss: 0.9492728114128113
train_loss: 0.9609916806221008; val_loss: 0.930394172668457
train_loss: 0.9933289885520935; val_loss: 0.944805383682251
train_loss: 0.9413076043128967; val_loss: 0.9356354475021362
train_loss: 0.9433019757270813; val_loss: 0.9481102228164673
train_loss: 0.936301589012146; val_loss: 0.9448279142379761
train_loss: 0.9295676350593567; val_loss: 0.9968221783638
train_loss: 0.93637013

In [ ]:
import torch
import numpy as np
from torch_geometric.data import Data


def filter_and_remap_edges(edge_index, node_mask):
    """
    Filter edges and remap node indices to consecutive integers.
    
    Args:
        edge_index: torch.Tensor of shape [2, E] 
        node_mask: torch.Tensor of shape [N,] with boolean values indicating valid nodes
    
    Returns:
        new_edge_index: torch.Tensor of shape [2, E'] with filtered and remapped edges
    """
    # Create mapping using cumsum (efficient)
    mapping = torch.cumsum(node_mask, dim=0) - 1
    mapping[~node_mask] = -1  # Mark invalid nodes
    
    # Filter and remap edges
    edge_mask = node_mask[edge_index[0]] & node_mask[edge_index[1]]
    filtered_edges = edge_index[:, edge_mask]
    
    # Apply mapping directly
    new_edge_index = mapping[filtered_edges]
    
    return new_edge_index


def filter_dataset(data: Data, mask: torch.Tensor) -> Data:
    """
    Filter dataset to keep only nodes where mask[i] = True.
    
    Args:
        data: PyTorch Geometric Data object
        mask: Boolean tensor of shape [N,] where N is the number of nodes
    
    Returns:
        train_data: New Data object with filtered nodes and remapped structure
    """
    
    # Ensure mask is boolean and on CPU for numpy operations
    mask = mask.bool()
    mask_np = mask.cpu().numpy()
    
    # Create new data object
    train_data = Data()
    
    n_units, n_attrs = data.n_units, data.n_attrs

    # 1. Filter node-level tensors
    if hasattr(data, 'x') and data.x is not None:
        train_data.x = torch.cat([data.x[:n_units][mask] , data.x[n_units:]], dim=0)
    
    if hasattr(data, 'treatment') and data.treatment is not None:
        train_data.treatment = data.treatment[mask]
    
    if hasattr(data, 'outcome') and data.outcome is not None:
        train_data.outcome = data.outcome[mask]
    
    if hasattr(data, 'true_effect') and data.true_effect is not None:
        train_data.true_effect = data.true_effect[mask]
    
    if hasattr(data, 'is_unit') and data.is_unit is not None:
        train_data.is_unit = torch.cat([data.is_unit[:n_units][mask] , data.is_unit[n_units:]], dim=0) #data.is_unit[mask]
    
    # 2. Filter mask tensors
    attrs_mask = mask.repeat_interleave(n_attrs)
    
    if hasattr(data, 'observed_mask') and data.observed_mask is not None:
        train_data.observed_mask = data.observed_mask[attrs_mask]
    
    if hasattr(data, 'treatment_mask') and data.treatment_mask is not None:
        train_data.treatment_mask = data.treatment_mask[mask]
    
    if hasattr(data, 'outcome_mask') and data.outcome_mask is not None:
        train_data.outcome_mask = data.outcome_mask[mask]
    
    # 3. Filter split masks
    if hasattr(data, 'train_mask') and data.train_mask is not None:
        train_data.train_mask = data.train_mask[mask]
    
    if hasattr(data, 'val_mask') and data.val_mask is not None:
        train_data.val_mask = data.val_mask[mask]
    
    if hasattr(data, 'test_mask') and data.test_mask is not None:
        train_data.test_mask = data.test_mask[mask]
    
    # 4. Filter and remap edge structures
    if hasattr(data, 'edge_index') and data.edge_index is not None:
        train_data.edge_index = filter_and_remap_edges(data.edge_index, mask)
        
        # Filter edge attributes if they exist
        if hasattr(data, 'edge_attr') and data.edge_attr is not None:
            edge_mask = mask[data.edge_index[0]] & mask[data.edge_index[1]]
            train_data.edge_attr = data.edge_attr[edge_mask]
    
    # 5. Filter relational edges
    if hasattr(data, 'rel_edge_index') and data.rel_edge_index is not None:
        train_data.rel_edge_index = filter_and_remap_edges(data.rel_edge_index, mask)
        
        # Filter relational edge types
        if hasattr(data, 'rel_edge_type') and data.rel_edge_type is not None:
            rel_edge_mask = mask[data.rel_edge_index[0]] & mask[data.rel_edge_index[1]]
            train_data.rel_edge_type = data.rel_edge_type[rel_edge_mask]
    
    # 6. Update metadata
    train_data.n_units = mask.sum().item()
    
    # Copy unchanged metadata
    for attr in ['n_attrs', 'n_rel_types', 'node_feature_dim', 'edge_attr_dim']:
        if hasattr(data, attr):
            setattr(train_data, attr, getattr(data, attr))
    
    # 7. Filter adjacency matrix (for network baselines)
    if hasattr(data, 'A') and data.A is not None:
        # Handle sparse tensor
        if hasattr(data.A, 'to_dense'):
            adj_dense = data.A.to_dense().cpu().numpy()
        else:
            adj_dense = data.A.cpu().numpy()
        
        # Filter adjacency matrix
        node_indices = np.where(mask_np)[0]
        filtered_adj = adj_dense[np.ix_(node_indices, node_indices)]
        
        # Convert back to same format as original
        if hasattr(data.A, 'to_dense'):
            # Was sparse, convert back to sparse
            train_data.A = torch.sparse_coo_tensor(
                indices=torch.nonzero(torch.tensor(filtered_adj)).t(),
                values=torch.tensor(filtered_adj)[torch.nonzero(torch.tensor(filtered_adj), as_tuple=True)],
                size=filtered_adj.shape
            ).to(data.A.device)
        else:
            train_data.A = torch.tensor(filtered_adj, device=data.A.device, dtype=data.A.dtype)
    
    # 8. Filter tabular data (numpy arrays)
    if hasattr(data, 'arr_X') and data.arr_X is not None:
        train_data.arr_X = data.arr_X[mask_np]
    
    if hasattr(data, 'arr_YF') and data.arr_YF is not None:
        train_data.arr_YF = data.arr_YF[mask_np]
    
    if hasattr(data, 'arr_Y1') and data.arr_Y1 is not None:
        train_data.arr_Y1 = data.arr_Y1[mask_np]
    
    if hasattr(data, 'arr_Y0') and data.arr_Y0 is not None:
        train_data.arr_Y0 = data.arr_Y0[mask_np]
    
    # Filter multi-dimensional adjacency matrix
    if hasattr(data, 'arr_Adj') and data.arr_Adj is not None:
        node_indices = np.where(mask_np)[0]
        if len(data.arr_Adj.shape) == 2:  # Single adjacency matrix
            train_data.arr_Adj = data.arr_Adj[np.ix_(node_indices, node_indices)]
        elif len(data.arr_Adj.shape) == 3:  # Multi-relational adjacency matrices
            train_data.arr_Adj = data.arr_Adj[:, np.ix_(node_indices, node_indices)]
    
    # 9. Filter dataframes
    if hasattr(data, 'df_full') and data.df_full is not None:
        train_data.df_full = data.df_full[mask_np].reset_index(drop=True)
    
    if hasattr(data, 'df_miss') and data.df_miss is not None:
        train_data.df_miss = data.df_miss[mask_np].reset_index(drop=True)
    
    if hasattr(data, 'df_imputed') and data.df_imputed is not None:
        train_data.df_imputed = data.df_imputed[mask_np].reset_index(drop=True)
    
    return train_data


# Example usage functions
def create_train_val_data(data: Data) -> Data:
    """
    Create dataset with only train and validation nodes.
    """
    mask = data.train_mask | data.val_mask
    return filter_dataset(data, mask)


def create_train_data_only(data: Data) -> Data:
    """
    Create dataset with only training nodes.
    """
    return filter_dataset(data, data.train_mask)


def create_test_data_only(data: Data) -> Data:
    """
    Create dataset with only test nodes.
    """
    return filter_dataset(data, data.test_mask)

import torch 


full_data_path = '/Users/jason/Documents/Coding Projects/2025_Claude/NetDeconf_main_hao/datasets/exps/BlogCatalog/p=0.0_k=9_seed=194.pt'
# full_data_path = 'datasets/exps/Syn/p=0.0_k=0_seed=919.pt'

data = torch.load(full_data_path, weights_only=False)
create_train_data_only(data)

IndexError: The shape of the mask [5196] at index 0 does not match the shape of the indexed tensor [5216, 20] at index 0

In [3]:
import torch

# Example inputs
mask = torch.tensor([True, False, True], dtype=torch.bool)  # shape (N,)
n_attrs = 4

# Expand mask
attrs_mask = mask.repeat_interleave(n_attrs)

print(attrs_mask)
# Output: tensor([ True,  True,  True,  True, False, False, False, False,  True,  True,  True,  True])

tensor([ True,  True,  True,  True, False, False, False, False,  True,  True,
         True,  True])


In [ ]:
import torch

def filter_and_remap_edges_v1(rel_edge_index, non_null_indicator):
    """
    Filter edges and remap node indices to consecutive integers.
    
    Args:
        rel_edge_index: torch.Tensor of shape [2, E] where each column represents an edge
        non_null_indicator: torch.Tensor of shape [N,] with boolean values indicating valid nodes
    
    Returns:
        new_rel_edge_index: torch.Tensor of shape [2, E'] with filtered and remapped edges
        node_mapping: torch.Tensor of shape [N,] mapping old indices to new indices (-1 for invalid nodes)
    """
    
    # Step 1: Create mapping from old indices to new consecutive indices
    # Only nodes with non_null_indicator[i] == True get new indices
    valid_nodes = torch.where(non_null_indicator)[0]  # indices where non_null_indicator is True
    
    # Create mapping: old_index -> new_index (-1 for invalid nodes)
    node_mapping = torch.full((non_null_indicator.size(0),), -1, dtype=torch.long)
    node_mapping[valid_nodes] = torch.arange(len(valid_nodes))
    
    # Step 2: Find which edges to keep
    # An edge is kept if both source and target nodes are valid (non_null_indicator == True)
    source_nodes = rel_edge_index[0]  # shape: [E]
    target_nodes = rel_edge_index[1]  # shape: [E]
    
    # Keep edge only if both source and target are valid
    keep_edge = non_null_indicator[source_nodes] & non_null_indicator[target_nodes]
    
    # Step 3: Filter edges and apply mapping
    filtered_source = node_mapping[source_nodes[keep_edge]]
    filtered_target = node_mapping[target_nodes[keep_edge]]
    
    new_rel_edge_index = torch.stack([filtered_source, filtered_target])
    
    return new_rel_edge_index, node_mapping


def filter_and_remap_edges(rel_edge_index, non_null_indicator):
    """
    Most optimized version - combines operations and minimizes memory allocations.
    """
    # Get valid node indices using cumsum (more efficient than torch.where for dense cases)
    valid_mask = non_null_indicator
    
    # Create compact mapping using cumsum
    mapping = torch.cumsum(valid_mask, dim=0) - 1
    mapping[~valid_mask] = -1  # Mark invalid nodes
    
    # Filter and remap in one step
    edge_mask = valid_mask[rel_edge_index[0]] & valid_mask[rel_edge_index[1]]
    filtered_edges = rel_edge_index[:, edge_mask]
    
    # Apply mapping directly
    new_rel_edge_index = mapping[filtered_edges]
    
    return new_rel_edge_index, mapping
# Example usage and test
if __name__ == "__main__":
    # Example: 5 nodes, some edges
    rel_edge_index = torch.tensor([
        [0, 1, 2, 3, 1, 4, 5, 2],  # source nodes
        [1, 2, 3, 4, 4, 0, 1, 5]   # target nodes
    ])
    
    # Mark nodes 1 and 3 as invalid (False), others as valid (True)
    non_null_indicator = torch.tensor([False, True, True, False, True, True])
    
    print("Original rel_edge_index:")
    print(rel_edge_index)
    print("\nNon-null indicator:", non_null_indicator)
    print("Valid nodes:", torch.where(non_null_indicator)[0].tolist())
    
    new_rel_edge_index, node_mapping = filter_and_remap_edges(rel_edge_index, non_null_indicator)
    
    print("\nNode mapping (old -> new):")
    for i, new_idx in enumerate(node_mapping):
        if new_idx != -1:
            print(f"  Node {i} -> Node {new_idx}")
        else:
            print(f"  Node {i} -> REMOVED")
    
    print(f"\nFiltered and remapped rel_edge_index:")
    print(new_rel_edge_index)
    print(f"Number of edges: {rel_edge_index.shape[1]} -> {new_rel_edge_index.shape[1]}")

In [ ]:
import torch
import numpy as np

def cartesian_embeddings_broadcast(node_emb, attr_emb):
    """The code to test."""
    N, M, K = node_emb.size(0), attr_emb.size(0), node_emb.size(1)
    
    # Use expand for memory efficiency (creates views, not copies)
    node_expanded = node_emb.unsqueeze(1).expand(N, M, K)
    attr_expanded = attr_emb.unsqueeze(0).expand(N, M, K)
    
    # Concatenate and reshape
    combined = torch.cat([node_expanded, attr_expanded], dim=2)
    return combined.view(N * M, 2 * K)

# Test 1: Basic functionality and shape verification
def test_basic_functionality():
    print("=== Test 1: Basic Functionality ===")
    N, M, K = 3, 2, 4
    
    node_emb = torch.randn(N, K)
    attr_emb = torch.randn(M, K)
    
    result = cartesian_embeddings_broadcast(node_emb, attr_emb)
    
    # Check shape
    expected_shape = (N * M, 2 * K)
    assert result.shape == expected_shape, f"Expected {expected_shape}, got {result.shape}"
    
    print(f"✓ Shape correct: {result.shape}")
    print(f"✓ Input shapes: node_emb={node_emb.shape}, attr_emb={attr_emb.shape}")

# Test 2: Value correctness - verify the indexing pattern
def test_value_correctness():
    print("\n=== Test 2: Value Correctness ===")
    N, M, K = 3, 2, 3
    
    # Create simple test data with known values
    node_emb = torch.arange(N * K, dtype=torch.float32).reshape(N, K)
    attr_emb = torch.arange(M * K, dtype=torch.float32).reshape(M, K) + 100
    
    print("Node embeddings:")
    print(node_emb)
    print("Attr embeddings:")
    print(attr_emb)
    
    result = cartesian_embeddings_broadcast(node_emb, attr_emb)
    
    print("\nResult:")
    print(result)
    
    # Verify specific entries
    # result[0] should be [node_emb[0], attr_emb[0]]
    expected_0 = torch.cat([node_emb[0], attr_emb[0]])
    assert torch.allclose(result[0], expected_0), f"Index 0 mismatch"
    
    # result[1] should be [node_emb[0], attr_emb[1]]
    expected_1 = torch.cat([node_emb[0], attr_emb[1]])
    assert torch.allclose(result[1], expected_1), f"Index 1 mismatch"
    
    # result[2] should be [node_emb[1], attr_emb[0]]
    expected_2 = torch.cat([node_emb[1], attr_emb[0]])
    assert torch.allclose(result[2], expected_2), f"Index 2 mismatch"
    
    print("✓ All indexing patterns correct")

# Test 3: Comprehensive indexing verification
def test_comprehensive_indexing():
    print("\n=== Test 3: Comprehensive Indexing ===")
    N, M, K = 4, 3, 2
    
    # Use sequential values for easy verification
    node_emb = torch.arange(N * K, dtype=torch.float32).reshape(N, K)
    attr_emb = torch.arange(M * K, dtype=torch.float32).reshape(M, K) + 1000
    
    result = cartesian_embeddings_broadcast(node_emb, attr_emb)
    
    # Verify every entry follows the pattern: result[n*M+m] = [node_emb[n], attr_emb[m]]
    for n in range(N):
        for m in range(M):
            idx = n * M + m
            expected = torch.cat([node_emb[n], attr_emb[m]])
            actual = result[idx]
            
            assert torch.allclose(actual, expected), \
                f"Mismatch at n={n}, m={m}, idx={idx}: expected {expected}, got {actual}"
    
    print(f"✓ All {N*M} combinations verified correctly")

# Test 4: Edge cases
def test_edge_cases():
    print("\n=== Test 4: Edge Cases ===")
    
    # Single node, multiple attributes
    node_emb = torch.randn(1, 5)
    attr_emb = torch.randn(10, 5)
    result = cartesian_embeddings_broadcast(node_emb, attr_emb)
    assert result.shape == (10, 10)
    print("✓ Single node case passed")
    
    # Multiple nodes, single attribute
    node_emb = torch.randn(8, 3)
    attr_emb = torch.randn(1, 3)
    result = cartesian_embeddings_broadcast(node_emb, attr_emb)
    assert result.shape == (8, 6)
    print("✓ Single attribute case passed")
    
    # Single node, single attribute
    node_emb = torch.randn(1, 4)
    attr_emb = torch.randn(1, 4)
    result = cartesian_embeddings_broadcast(node_emb, attr_emb)
    assert result.shape == (1, 8)
    print("✓ Single-single case passed")

# Test 5: Large tensor performance
def test_large_tensors():
    print("\n=== Test 5: Large Tensor Performance ===")
    import time
    
    N, M, K = 1000, 500, 128
    node_emb = torch.randn(N, K)
    attr_emb = torch.randn(M, K)
    
    start_time = time.time()
    result = cartesian_embeddings_broadcast(node_emb, attr_emb)
    end_time = time.time()
    
    expected_shape = (N * M, 2 * K)
    assert result.shape == expected_shape
    
    print(f"✓ Large tensor test passed: {result.shape}")
    print(f"✓ Time taken: {end_time - start_time:.3f} seconds")

# Test 6: Gradient preservation
def test_gradient_preservation():
    print("\n=== Test 6: Gradient Preservation ===")
    
    N, M, K = 3, 2, 4
    node_emb = torch.randn(N, K, requires_grad=True)
    attr_emb = torch.randn(M, K, requires_grad=True)
    
    result = cartesian_embeddings_broadcast(node_emb, attr_emb)
    
    # Compute a simple loss
    loss = result.sum()
    loss.backward()
    
    assert node_emb.grad is not None, "Node embedding gradients not computed"
    assert attr_emb.grad is not None, "Attribute embedding gradients not computed"
    assert node_emb.grad.shape == node_emb.shape, "Node gradient shape mismatch"
    assert attr_emb.grad.shape == attr_emb.shape, "Attribute gradient shape mismatch"
    
    print("✓ Gradients computed correctly")
    print(f"✓ Node grad shape: {node_emb.grad.shape}")
    print(f"✓ Attr grad shape: {attr_emb.grad.shape}")

# Test 7: Different data types and devices
def test_dtypes_and_devices():
    print("\n=== Test 7: Data Types and Devices ===")
    
    N, M, K = 2, 3, 4
    
    # Test different dtypes
    for dtype in [torch.float32, torch.float64, torch.float16]:
        node_emb = torch.randn(N, K, dtype=dtype)
        attr_emb = torch.randn(M, K, dtype=dtype)
        result = cartesian_embeddings_broadcast(node_emb, attr_emb)
        
        assert result.dtype == dtype, f"Dtype not preserved: expected {dtype}, got {result.dtype}"
        assert result.shape == (N * M, 2 * K)
    
    print("✓ All data types work correctly")
    
    # Test GPU if available
    if torch.cuda.is_available():
        device = torch.device('cuda')
        node_emb = torch.randn(N, K, device=device)
        attr_emb = torch.randn(M, K, device=device)
        result = cartesian_embeddings_broadcast(node_emb, attr_emb)
        
        assert result.device == device, "Device not preserved"
        assert result.shape == (N * M, 2 * K)
        print("✓ GPU test passed")
    else:
        print("✓ GPU not available, skipping GPU test")

# Test 8: Memory efficiency verification
def test_memory_efficiency():
    print("\n=== Test 8: Memory Efficiency ===")
    
    N, M, K = 100, 50, 64
    node_emb = torch.randn(N, K)
    attr_emb = torch.randn(M, K)
    
    # Get initial memory
    initial_memory = torch.cuda.memory_allocated() if torch.cuda.is_available() else 0
    
    # Create intermediate tensors to verify they use views
    node_expanded = node_emb.unsqueeze(1).expand(N, M, K)
    attr_expanded = attr_emb.unsqueeze(0).expand(N, M, K)
    
    # Verify that expand creates views (shares storage)
    assert node_expanded.storage().data_ptr() == node_emb.storage().data_ptr(), \
        "expand() should create views, not copies"
    
    print("✓ Memory efficiency verified - expand() creates views")

# Test 9: Comparison with naive implementation
def test_comparison_with_naive():
    print("\n=== Test 9: Comparison with Naive Implementation ===")
    
    def naive_cartesian(node_emb, attr_emb):
        """Naive nested loop implementation for comparison."""
        N, M, K = node_emb.size(0), attr_emb.size(0), node_emb.size(1)
        result = torch.zeros(N * M, 2 * K, dtype=node_emb.dtype, device=node_emb.device)
        
        for n in range(N):
            for m in range(M):
                idx = n * M + m
                result[idx] = torch.cat([node_emb[n], attr_emb[m]])
        
        return result
    
    N, M, K = 5, 4, 6
    node_emb = torch.randn(N, K)
    attr_emb = torch.randn(M, K)
    
    result_broadcast = cartesian_embeddings_broadcast(node_emb, attr_emb)
    result_naive = naive_cartesian(node_emb, attr_emb)
    
    assert torch.allclose(result_broadcast, result_naive, atol=1e-6), \
        "Results don't match between broadcast and naive implementation"
    
    print("✓ Results match naive implementation perfectly")

# Run all tests
if __name__ == "__main__":
    tests = [
        test_basic_functionality,
        test_value_correctness,
        test_comprehensive_indexing,
        test_edge_cases,
        test_large_tensors,
        test_gradient_preservation,
        test_dtypes_and_devices,
        test_memory_efficiency,
        test_comparison_with_naive
    ]
    
    print("Running comprehensive tests for cartesian embeddings...\n")
    
    for test in tests:
        try:
            test()
        except Exception as e:
            print(f"❌ {test.__name__} FAILED: {e}")
            break
    else:
        print("\n🎉 All tests passed! The implementation is correct.")